In [31]:
import pandas as pd
import numpy as np
import datetime
import copy
import json
import jenkspy

In [25]:
data = pd.read_csv('./avif_dump_4940.csv', encoding='utf-8', low_memory=False)

In [21]:
data.columns

Index(['Unnamed: 0', 'Activity', 'Age', 'AllSecret', 'AllSecretNow', 'Closed',
       'Club300Rarity', 'CoObservers', 'CommentsCount', 'Complete',
       'CountExMiddle', 'CountExact', 'CountHigh', 'CountLow', 'DateCr',
       'DateUpd', 'Group', 'Id', 'IsCap', 'IsCapcerny', 'IsCejka',
       'IsHourlist', 'IsInteresting', 'IsIwc', 'IsStrnad', 'KODCOBE_',
       'KODOK_', 'LastUpdBy', 'Latitude', 'Longitude', 'Lsd', 'MarkNo',
       'Mobile', 'MyFirstThisYear', 'NAZCOBE', 'NAZKR', 'NAZOB', 'NAZOK',
       'NK_', 'Note', 'ObsDate', 'ObsDate_fmt', 'ObsDate_present', 'ObsListId',
       'ObsListNote', 'ObsListRecOwner', 'ObsOwner', 'ObsOwnerFullName',
       'ObsOwnerName', 'ObsOwnerSurname', 'OtherActivity', 'OtherSpecies',
       'PhotoId', 'PrecisionScale', 'RecOwner', 'SecretUntil',
       'SecretUntil_fmt', 'Sex', 'SiteName', 'SiteNote', 'SiteSecret',
       'SiteSecretLevel', 'SiteSecretNow', 'SiteSecretNowForMe', 'Species',
       'SpeciesId', 'SpeciesPresent', 'SpeciesSci', 'Speci

In [28]:
data = data[data.Complete == 1]

In [4]:
data.dropna(subset=['ySquareX', 'ySquareY', 'SpeciesId'], inplace=True)

In [5]:
data.SpeciesId = data.SpeciesId.astype(int).astype(str)

In [6]:
data['Week'] = data.ObsDate.apply(lambda x: int(datetime.datetime.strptime(x, "%Y-%m-%d" ).strftime("%V")))

In [7]:
data['Square'] = data.ySquareY.astype(int).astype(str) + data.ySquareX.astype(int).astype(str)

In [8]:
# dump species list
with open('../speclist.js', 'w', encoding='utf-8') as f:
    f.write('var specList = ' 
            + json.dumps(data[['SpeciesId', 'SpeciesSci', 'Species']].set_index('SpeciesId').to_dict(orient='index')) 
            + ';')

In [9]:
out = {}
for sq in data.Square.unique():
    if sq not in out:
        out[sq] = {}
    for sp in data.SpeciesId.unique():
        out[sq][sp] = [0] * 52
    out[sq]['wks'] = [0] * 52

In [10]:
for sqr in out:
    sub = data[data.Square == sqr]
    for w in sub.groupby(['Week']).ObsListId.unique().reset_index().values:
        out[sqr]['wks'][w[0] - 1] = len(w[1])
        
    for specie in list(sub.SpeciesId.unique()):
        for w in sub[sub.SpeciesId == specie].groupby(['Week']).ObsListId.unique().reset_index().values:
            out[sqr][specie][w[0] - 1] = len(w[1])

In [11]:
# podil druhu na listech
for sqr in out:
    for key in out[sqr]:
        if key == 'wks':
            continue
        out[sqr][key] = list(np.divide(out[sqr][key], out[sqr]['wks']))

c:\program files\python36\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in true_divide
  


In [12]:
# likvidace nanu a nekonecen
def cln(val):
    if np.isfinite(val) == True:
        return val
    else:
        return 0
    
clean = np.vectorize(cln, otypes=[np.float])

for sqr in out.values():
    for key in sqr:
        sqr[key] = list(clean(sqr[key]))

In [13]:
#del prazdnych klicu
out_cp = copy.deepcopy(out)
for sqr in out_cp:
    for key in out_cp[sqr]:
        if key == 'wks':
            continue
        if sum(out_cp[sqr][key]) == 0:
            del out[sqr][key]

In [14]:
for sq in out:
    tmp = out[sq]
    for t in tmp: # Numpy JSON vohack
        tmp[t] = list(map(float, tmp[t]))
    with open('../stats/' + str(sq) + '.json', 'w', encoding='utf-8') as f:
        f.write(json.dumps(tmp))

In [15]:
stats = {'sum': 0}
for sq in out:
    stats[sq] = sum(out[sq]['wks'])
    stats['sum'] += sum(out[sq]['wks'])

In [16]:
with open('../stats/stats.js', 'w', encoding='utf-8') as f:
    f.write('var stats = ' + json.dumps(stats) + ';')

In [35]:
o = []
for s in list(stats.values())[1:]:
    o.append((s / stats['sum']))

In [36]:
breaks = jenkspy.jenks_breaks(o, nb_class=5)